In [237]:
import pandas as pd
from gurobipy import *

xls = pd.ExcelFile('Proyecto_1_TransporteMaritimo_Dataset1.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])
df6= pd.read_excel(xls, nombres[5])

In [322]:
n_cargos = len(df3)

In [323]:
V= list(df1["ID_Barco"]) ## conjunto barcos
n_barcos = len(V)

In [324]:

Kv= {} # conjunto capacidades de barcos
for i in range(0,len(V)):
    Kv[i+1]= list(df1["Capacidad"])[i]

In [325]:
nodos_carga= list(df3["ID_Cargo"])

nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga

In [326]:
cargas = df3[["ID_Cargo","ID_Puerto_Origen", "Tamano", "Costo_SPOT","LT_Carga","RT_Carga"]].rename(columns={"ID_Puerto_Origen": "ID_Puerto ", "LT_Carga":"LT","RT_Carga":"RT"})
descargas = df3[["ID_Cargo","ID_Puerto_Destino", "Tamano", "Costo_SPOT","LT_Descarga","RT_Descarga"]].rename(columns={"ID_Puerto_Destino": "ID_Puerto ", "LT_Descarga":"LT","RT_Descarga":"RT"})
nodos = pd.concat([cargas,descargas],axis=0)
nodos = nodos.reset_index()[["ID_Cargo","ID_Puerto ", "Tamano", "Costo_SPOT","LT","RT"]]
nodos.index+=1
N = nodos.to_dict("index")

puertos_barco = df1["ID_Puerto "]
t_origen = df1["Tiempo_Inicio"].to_dict()
o = {}
d = {}
for i in range(len(puertos_barco.to_dict())):
  o[i+1] = i+1+n_cargos*2 
  N[i+1+n_cargos*2] = {'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': puertos_barco.to_dict()[i],
  'LT': t_origen[i],
  'RT': t_origen[i],
  'Tamano': 0}
  d[i+1]=i+1+n_cargos*2+len(puertos_barco.to_dict())
  N[i+1+n_cargos*2+len(puertos_barco.to_dict())]={'Costo_SPOT': 0,
  'ID_Cargo': 0,
  'ID_Puerto ': 0,
  'LT':1388,
  'RT': 1388,
  'Tamano': 0}
nodos_index = [i for i in N.keys()]

In [327]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
  Nv[i] = []
  for j in N.keys():
    if N[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
      Nv[i].append(j)


In [568]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in N[v]:
        for j in N[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)
print(Av[1])

[('ID_Cargo', 'ID_Cargo'), ('ID_Cargo', 'ID_Puerto '), ('ID_Cargo', 'Tamano'), ('ID_Cargo', 'Costo_SPOT'), ('ID_Cargo', 'LT'), ('ID_Cargo', 'RT'), ('ID_Puerto ', 'ID_Cargo'), ('ID_Puerto ', 'ID_Puerto '), ('ID_Puerto ', 'Tamano'), ('ID_Puerto ', 'Costo_SPOT'), ('ID_Puerto ', 'LT'), ('ID_Puerto ', 'RT'), ('Tamano', 'ID_Cargo'), ('Tamano', 'ID_Puerto '), ('Tamano', 'Tamano'), ('Tamano', 'Costo_SPOT'), ('Tamano', 'LT'), ('Tamano', 'RT'), ('Costo_SPOT', 'ID_Cargo'), ('Costo_SPOT', 'ID_Puerto '), ('Costo_SPOT', 'Tamano'), ('Costo_SPOT', 'Costo_SPOT'), ('Costo_SPOT', 'LT'), ('Costo_SPOT', 'RT'), ('LT', 'ID_Cargo'), ('LT', 'ID_Puerto '), ('LT', 'Tamano'), ('LT', 'Costo_SPOT'), ('LT', 'LT'), ('LT', 'RT'), ('RT', 'ID_Cargo'), ('RT', 'ID_Puerto '), ('RT', 'Tamano'), ('RT', 'Costo_SPOT'), ('RT', 'LT'), ('RT', 'RT')]


In [569]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]



In [570]:
LT_i = {i:N[i]["LT"] for i in N.keys()}
RT_i = {i:N[i]["RT"] for i in N.keys()}

In [571]:
T_ijv={}
for i in N.keys():
    T_ijv[i] = {} 
    for j in N.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= N[i]["ID_Cargo"]
            cargo_j = N[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                T_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Tiempo_Viaje_hrs"]]["Tiempo_Viaje_hrs"].item()
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Origen_hrs"]]["Tiempo_Origen_hrs"].item()
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Tiempo_Destino_hrs"]]["Tiempo_Destino_hrs"].item()
                if ((i,j) not in Av[v]) or j==i:
                    T_ijv[i][j][v] += 10000000

In [572]:
T_ijv[33][15][3]

10000000

In [ ]:
C_ijv={}
for i in N.keys():
    C_ijv[i] = {} 
    for j in N.keys():
        C_ijv[i][j] = {} 
        for v in V:
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = N[i]["ID_Puerto "]
                puerto_destino = N[j]["ID_Puerto "]
                C_ijv[i][j][v] = df4.loc[(df4["Puerto_Origen"]==puerto_origen)&(df4["Puerto_Destino"]==puerto_destino)&(df4["ID_Barco"]==v),["Costo_Viaje_libras"]]["Costo_Viaje_libras"].item()
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Origen_hrs"]]["Costos_Origen_hrs"].item()
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += df5.loc[(df5["ID_Barco"]==v)&(df5["ID_Cargo"]==N[i]["ID_Cargo"]),["Costos_Destino_hrs"]]["Costos_Destino_hrs"].item()
                if ((i,j) not in Av[v]) or j==i:
                    C_ijv[i][j][v] += 10000000

In [ ]:
C_ijv[31][31][1]

In [ ]:
Qj={N[i]["ID_Cargo"]:N[i]["Tamano"] for i in N.keys() if N[i]["ID_Cargo"]>0}
print(Qj)

Cs = df3["Costo_SPOT"].to_dict()
print("empezando con el modelo")
print(nodos_carga)

In [ ]:

# Model
m = Model("Transporte Maritimo")

# variables

#x = m.addVars(indices, vtype=GRB.BINARY, name="Arco")
x = m.addVars(nodos_index, nodos_index, V, vtype=GRB.BINARY, name="Arco")
y = m.addVars(nodos_carga,vtype=GRB.BINARY,name="Cargo_Spot")
t = m.addVars(nodos_index,V,vtype=GRB.INTEGER, name="tiempo")
l = m.addVars(nodos_index,V,vtype=GRB.INTEGER,name="Inventario")

In [ ]:
# Objective function
m.setObjective(sum(sum(C_ijv[i][j][v]*x[i,j,v] for (i,j) in Av[v]) for v in V)+sum(Cs[i-1]*y[i] for i in nodos_carga), GRB.MINIMIZE)

In [ ]:
m.addConstrs(((sum(sum(x[i,j,v] + y[i] for j in Nv[v]) for v in V) == 1) for i in nodos_carga), name = "Const_1")
#m.addConstrs(((sum(sum(x[i,j,v] + y[i] for j in Nv[v] if (i,j) in Av[v]) for v in V) == 1) for i in nodos_carga), name = "Const_1")

In [ ]:
m.addConstrs(((sum(x[o[v],j,v] for j in Nv[v]) == 1) for v in V), name = "Const_2")

In [ ]:
m.addConstrs((((sum(x[i,j,v] for j in Nv[v])-sum(x[j,i,v] for j in Nv[v]))==0) for v in V for i in Nv[v] if i<=n_cargos*2), name = "Const_3")

In [ ]:
m.addConstrs(((sum(x[j,d[v],v] for j in Nv[v]) == 1) for v in V), name = "Const_4")

In [ ]:
m.addConstrs(((l[i,v]+Qj[j]-l[j,v]<=Kv[v]*(1-x[i,j,v])) for v in V  for j in NP_v[v] for i in Nv[v] if ((i,j) in Av[v])), name = "Const_5")

In [ ]:
m.addConstrs(((l[i,v]-Qj[j]-l[n_cargos+j,v]<=Kv[v]*(1-x[i,n_cargos+j,v]))  for v in V for j in NP_v[v] for i in Nv[v] if ((i,j+n_cargos) in Av[v])), name = "Const_6")

In [ ]:
m.addConstrs(((l[i,v]<=Kv[v]) for v in V for i in NP_v[v]), name = "Const_7_ub")

In [ ]:
m.addConstrs(((l[i,v]>=0) for v in V for i in NP_v[v]), name = "Const_7_lb")

In [ ]:
m.addConstrs(((t[i,v]+T_ijv[i][j][v]-t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-x[i,j,v])) for v in V for (i,j) in Av[v]), name = "Const_8")

In [ ]:
m.addConstrs(((sum(x[i,j,v] for j in Nv[v])-sum(x[n_cargos+i,j,v] for j in Nv[v])==0) for v in V for i in NP_v[v]), name = "Const_9")

In [ ]:

m.addConstrs(((t[i,v]+T_ijv[i][n_cargos+i][v]-t[n_cargos+i,v]<=0) for v in V for i in NP_v[v] ), name = "Const_10")

In [ ]:
m.addConstrs(((t[i,v]<=RT_i[i]) for v in V for i in Nv[v]), name = "Const_11_ub")

In [ ]:
h = m.addConstrs(((LT_i[i]<=t[i,v]) for v in V for i in Nv[v]), name = "Const_11_lb")

In [ ]:
# Solve
m.optimize()

valorobjetivo = m.objval

"""for v in m.getVars():
    print('%s %g' % (v.varName, v.x))"""

print('\n Valor óptimo: %8.4f \n' % valorobjetivo)

m.write("Transporte_final.sol")
m.write("model.lp")

In [ ]:
#x[31,31,1].Xn

In [ ]:
for v in V:
    for i in nodos_index:
        for j in nodos_index:
            if x[i,j,v].X==1:
                print(f"x[{i},{j},{v}]")